# I could make pivot tables in excel and be done but this is a chance to show off some advanced data skills

# Load some libraries

In [44]:
import pandas as pd
import numpy as np
import ibm_db
import ibm_db_dbi

# Setup some connection parameters to my database on IBM Cloud
#### I loaded the 2020 csv into my database using their interface

In [4]:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net"
dsn_uid = "nbv31807"
dsn_pwd = "8jrp5+fwlqxprl9j"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "50000" 
dsn_protocol = "TCPIP"

In [5]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

# Connect

In [7]:
try:
    con = ibm_db.connect(dsn, "", "")
    print("connected")
except:
    print("borked")

connected


In [8]:
try:
    pcon = ibm_db_dbi.connect(dsn)
    print("connected")
except:
    print("borked")

connected


# Read the TS20 Advanced Action Listings CSV into a Dataframe

In [33]:
df_main=pd.read_sql('select * from TS20_Listings', pcon)
pd.set_option('display.max_columns', None)

In [56]:
df_main.head()

,Action_Date,Action_Id,OID,Status,Revenue,Action_Cost,Rate,Promo_Code,Partner,Partner_Id,Partner_Group,Event_Type,Event_Type_Id,Ad,Ad_Id,Ad_Type,SubId1,SubId2,SubId3,Shared_Id,Referral_Id,Indirect_Tax,Year
0,2020-04-15 23:59:51,5683.4488.547833,0e106cc5-dd8e-4671-81a0-bd33bffb59b4,Pending,0.00,0.00,0.00,None,NameSpace,18896,Main: Technology,Account registration,12723,Online Tracking Link,391260,ONLINE_TRACKING_LINK,None,None,None,None,C39e8825f-7796-11ea-8e10-42010a246f1e/18896/39...,0.0,2020
1,2020-04-15 23:59:49,5683.4488.578195,g5cc897ggd6842d4,Pending,0.00,0.00,0.00,None,NameSpace,18896,Main: Technology,Online Sale,12724,Online Tracking Link,391260,ONLINE_TRACKING_LINK,None,None,None,None,Ce69721a0-6f8c-11ea-bfe4-42010a246c13/18896/39...,0.0,2020
2,2020-04-15 23:59:49,5683.4488.547836,334adad9f4634f2b,Pending,94.98,20.90,0.22,None,Ziff Davis - Offers.com,10098,"Main: Coupon, Sub: TM+",Online Sale,12724,Save 35% on H&R Block Software Products!,531484,TEXT_LINK,None,None,None,None,Ca6380fb0-5e2a-11e9-867d-0699cc181ccb/10098/53...,0.0,2020
3,2020-04-15 23:59:21,5683.4488.578189,52hhdh4f2c5g4d8c,Pending,82.48,14.85,0.18,None,"Ebates Performance Marketing, Inc. dba Rakuten...",10579,"Main: Loyalty, Sub: Typo Squatter/Redirector",Online Sale,12724,H&R Block Homepage,467573,TEXT_LINK,None,None,None,None,Caf34c273-4bac-11ea-ab8c-42010a246e0e/10579/46...,0.0,2020
4,2020-04-15 23:59:18,5683.4488.547799,fd795325he2444fb,Pending,187.47,30.00,0.16,None,MaxBounty.com ULC,10813,"Main: Sub Network, Sub: New Recruit 2018",Online Sale,12724,H&R Block Homepage,467573,TEXT_LINK,None,None,None,449034,Cb9065fd2-7f82-11ea-9e06-42010a246c12/10813/46...,0.0,2020


# Add a simple year column for use in the next stage

In [35]:
year = []
for row in df_main["Action_Date"]:
    if row.year > 2000:
        year.append(row.year)
df_main["Year"] = year

# Convert the Dataframe into a pivot table

In [58]:
df_pivot = df_main.pivot_table(
    values='Revenue',
    index=["Year","Partner_Group","Partner"],
    columns='Event_Type',
    aggfunc=np.sum)
df_pivot

Event_Type                                                                       Account registration  \
Year Partner_Group                                      Partner                                         
2020 Main: Content Sites                                Ad Practitioners Main                     0.0   
                                                        Business Insider.                         0.0   
                                                        CBS Interactive Inc.                      0.0   
                                                        Clark Howard Inc.                         0.0   
                                                        Cover Story Media, Inc.                   0.0   
...                                                                                               ...   
     Main: Technology, Sub: New Recruits 2020           symbaloonl                                0.0   
     Sub: New Recruit 2017, Main: Loyalty               Wikibuy                                   0.0   
     Sub: New Recruit 2018, Main: Technology            Guppy Games, Inc.                         0.0   
     Sub: New Recruits 2020, Main: Technology           CollegeBacker                             0.0   
     Sub: New Recruits 2020, Sub: Typo Squatter/Redi... Resilion, LLC.                            0.0   

Event_Type                                                                       In-store Purchase  \
Year Partner_Group                                      Partner                                      
2020 Main: Content Sites                                Ad Practitioners Main                  NaN   
                                                        Business Insider.                      NaN   
                                                        CBS Interactive Inc.                   NaN   
                                                        Clark Howard Inc.                      NaN   
                                                        Cover Story Media, Inc.                NaN   
...                                                                                            ...   
     Main: Technology, Sub: New Recruits 2020           symbaloonl                             NaN   
     Sub: New Recruit 2017, Main: Loyalty               Wikibuy                                NaN   
     Sub: New Recruit 2018, Main: Technology            Guppy Games, Inc.                      NaN   
     Sub: New Recruits 2020, Main: Technology           CollegeBacker                          NaN   
     Sub: New Recruits 2020, Sub: Typo Squatter/Redi... Resilion, LLC.                         NaN   

Event_Type                                                                       Online Sale  \
Year Partner_Group                                      Partner                                
2020 Main: Content Sites                                Ad Practitioners Main        1428.38   
                                                        Business Insider.           11542.66   
                                                        CBS Interactive Inc.        17774.05   
                                                        Clark Howard Inc.             164.46   
                                                        Cover Story Media, Inc.       690.27   
...                                                                                      ...   
     Main: Technology, Sub: New Recruits 2020           symbaloonl                    816.09   
     Sub: New Recruit 2017, Main: Loyalty               Wikibuy                    977263.82   
     Sub: New Recruit 2018, Main: Technology            Guppy Games, Inc.             546.00   
     Sub: New Recruits 2020, Main: Technology           CollegeBacker                   0.00   
     Sub: New Recruits 2020, Sub: Typo Squatter/Redi... Resilion, LLC.             241151.10   

Event_Type                                                

# Saving the pivot as a csv

In [59]:
df_pivot.to_csv("TS20_Listings.csv")
